In [18]:
import json
import pandas as pd
from datetime import datetime
from datetime import timedelta
import datetime
pd.set_option('display.max_rows', None)

In [19]:
#Loading the User Submitted JSON Data

# Define the file path
file_path = "json_data.json"

# Read and parse the JSON file
try:
    with open(file_path, 'r') as file:
        user_submitted_data = json.load(file)
        # print(user_submitted_data)
except FileNotFoundError:
    print(f"The file at path {file_path} was not found.")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")


In [20]:
#Initialize Columns
columns = ['Stage',"Fabric Name", 'Process', 'Description', 'Quantity of Material', 'Color', 'Original Start Date', 'Lowest Days', 'Original End Date']
df_workflow_tracker = pd.DataFrame(columns=columns)

# Adding Final Calculated Data into Dataframe
def add_to_dataframe(df, stage, fabric_name, process, description, quantity, color, start_date, lowest_days, end_date):
    new_row = pd.DataFrame([{
        'Stage': stage,
        'Fabric Name': fabric_name,
        'Process': process,
        'Description': description,
        'Quantity of Material': quantity,
        'Color': color,
        'Original Start Date': start_date,
        'Lowest Days': lowest_days,
        'Original End Date': end_date
    }])
    return pd.concat([df, new_row], ignore_index=True)

In [21]:
#Calculating the Days In Hand:

def string_to_datetime_converter(date_string):
    return datetime.datetime.strptime(date_string, '%Y-%m-%d')

def calculate_days_in_hand():
    user_submitted_data["shipDate"] = string_to_datetime_converter(user_submitted_data["shipDate"])
    user_submitted_data["orderDate"] = string_to_datetime_converter(user_submitted_data["orderDate"])
    days_in_hand = (user_submitted_data["shipDate"]-user_submitted_data["orderDate"]).days 
    return days_in_hand

calculate_days_in_hand()


58

In [22]:
#Calculating the total order Quantity. 
def total_order_quantity():
   total_order_quantity = 0

   for fabric_details in user_submitted_data["fabricDetails"]:
      fabric_total_quantity = 0
      for color_quantity in fabric_details["colors"]:
         fabric_total_quantity+=int(color_quantity["quantity"])
   total_order_quantity=max(total_order_quantity,fabric_total_quantity)

In [23]:
#Handling the PURCHASE Section


def fetch_supporting_data(location):
     return pd.read_excel(location)

df_procurement_days_supporter = fetch_supporting_data("Excel Files/Purchase.xlsx")


def get_procurement_days(Material, quantity):
    # Filter the DataFrame based on the fabric
    fabric_data = df_procurement_days_supporter[df_procurement_days_supporter['Material'] == Material]
    
    # Iterate through the rows to find the matching quantity range
    for _, row in fabric_data.iterrows():
        if row['Quantity Minimum'] <= quantity <= row['Quantity Maximum']:
            return row['Minimum days']
    
    return "Quantity out of range or fabric not found"

class FabricColorAndQuantity:
    def  __init__(self, fabric_name):
        self.fabric_name = fabric_name
        self.__fabric_all_data = self.__get_fabric_data()
        self.total_fabric_quantity = self.__calculate_total_quantity()
        self.__material_required_per_piece_of_garment = float(self.__fabric_all_data['usage']['value'])
        
    def __get_fabric_data(self):
        for fabric_data in user_submitted_data["fabricDetails"]:
            if self.fabric_name == fabric_data["name"]:
                return fabric_data

    def __calculate_total_quantity(self):
                return sum([int(x['quantity']) for x in self.__fabric_all_data['colors']])
            
    def get_quantity_for_one_color(self,color_name):     
                for color in self.__fabric_all_data['colors']:
                    if color_name==color['name']:
                        return int(color['quantity'])
                    
    def total_material_required(self,*args):
         if len(args)==0:
              return self.total_fabric_quantity * self.__material_required_per_piece_of_garment
         return self.get_quantity_for_one_color(args[0])*self.__material_required_per_piece_of_garment

            
def end_date_calculator(start_date, number_of_days):
    # Helper function to convert string to date
    def string_to_date(date_str):
        return datetime.strptime(date_str, '%Y-%m-%d').date()

    # Convert start_date to date if it's a string
    if isinstance(start_date, str):
        start_date = string_to_date(start_date)

    # Convert number_of_days to an integer if it's a string
    if isinstance(number_of_days, str):
        number_of_days = int(number_of_days)
    elif isinstance(number_of_days, timedelta):
        number_of_days = number_of_days.days

    # Calculate end date
    end_date = start_date + timedelta(days=number_of_days)

    return end_date
     

# Example usage
for material_details in user_submitted_data["fabricDetails"]:
    material_name =  material_details["name"]

    total_fabric_quantity=  FabricColorAndQuantity(material_name).total_fabric_quantity                         

    procurement_days = get_procurement_days(material_name,total_fabric_quantity)
    end_date =  end_date_calculator(user_submitted_data["orderDate"], procurement_days)


    df_workflow_tracker=add_to_dataframe(
        df_workflow_tracker,  #This can be changed to by default df_workflow_tracker
        "Purchase",
        material_name,
        "GREIGE",
        f"Buying  {material_name}",
        FabricColorAndQuantity(material_name).total_material_required(),
        "null",
        user_submitted_data["orderDate"],
        procurement_days,
        end_date  # This shouldn't be calculated here, it has to be done inside the parent function.
    )

    df_workflow_tracker

C:\Users\DKC\AppData\Local\Temp\ipykernel_17068\4100248132.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, new_row], ignore_index=True)


In [24]:
#Handling Submission Section this code is right now only handling the chinese fabric. 

df_submission_days_supporter = fetch_supporting_data("Excel Files/submission.xlsx")

def get_submission_days(process): # This function will return the submission days for any process.
    process_data =  df_submission_days_supporter[df_submission_days_supporter['Process'] == process]
    return int(process_data["Minimum days"].iloc[0])



class FabricWithSimilarProcess:  # This class will return the list of all fabrics, that follow a predefined process. This has been extensively, used for Mock Up (Shiffly) Identification.

    def  __init__(self, process):
        self.process =  process
        self.fabric_list = self.__get_fabric_list()

        def __get_fabric_list(self) -> list:
            fabric_list =[]
            for fabric_data in user_submitted_data["fabricDetails"]:
                if self.process in  fabric_data["processes"]:
                    fabric_list.append(fabric_data['name'])
            
            return fabric_list


china_fabric_contains = user_submitted_data["china_fabric_present"] == "Yes"

#Only Handling the case of china fabric.


#Handling case 1 : Submission. 

def fabric_processes_list(fabric_name):
    data = user_submitted_data["fabricDetails"]
    for fabric in data:
        if data['name'] == fabric:
            return  data['processes']

def handling_shiffly_for_all_fabrics_in_case_china_fabric_is_present(fabric_to_be_processed_in_this_stage,start_date,delay_by_days):

    for fabric in fabric_to_be_processed_in_this_stage:
        lowest_days_needed=int(df_submission_days_supporter[df_submission_days_supporter["Process"]=='SHIFFLY']['Minimum days'].iloc[0])
        start_date_china_shiffly = start_date +timedelta(days=delay_by_days)
        end_date_china_shiffly = end_date_calculator(start_date_china_shiffly,lowest_days_needed)
        df_workflow_tracker = add_to_dataframe(
            df_workflow_tracker,
            stage="Submission",
            fabric_name=fabric,
            process="SHIFFLY",
            description=f"SHIFFLY process for {fabric}",
            quantity=total_fabric_quantity,
            color="null",
            start_date=start_date_china_shiffly,
            lowest_days=lowest_days_needed,
            end_date=end_date_china_shiffly
        ) 


def handling_dying_for_fabric():
    pass
    

if china_fabric_contains:

    fabric_to_be_processed_in_this_stage = FabricWithSimilarProcess("SHIFFLY").fabric_list 
    handling_shiffly_for_all_fabrics_in_case_china_fabric_is_present(fabric_to_be_processed_in_this_stage,user_submitted_data["orderDate"],1)

else:
    major_fabric_name = user_submitted_data["major_fabric"]
    major_fabric_processes =  fabric_processes_list(major_fabric_name)
    if  "PRINTING" in major_fabric_processes:
        pass
    elif "SHIFFLY" in  major_fabric_processes:
         handling_shiffly_for_all_fabrics_in_case_china_fabric_is_present([major_fabric_name],user_submitted_data["orderDate"],1)   
         if "DYING" in major_fabric_processes:
             pass
            
#Functions to be used. 
df_workflow_tracker

<class 'bool'> True


,Stage,Fabric Name,Process,Description,Quantity of Material,Color,Original Start Date,Lowest Days,Original End Date
0,Purchase,WINDOWPANE GAUZ,GREIGE,Buying WINDOWPANE GAUZ,12000.0,null,2024-09-03,10,2024-09-13
1,Purchase,CHINA LACE,GREIGE,Buying CHINA LACE,3840.0,null,2024-09-03,30,2024-10-03
2,Purchase,FP001,GREIGE,Buying FP001,6000.0,null,2024-09-03,20,2024-09-23
3,Submission,FP001,SHIFFLY,SHIFFLY process for FP001,12000.0,null,2024-09-04,10,2024-09-14


In [25]:
df_fob_days_supporter = pd.read_excel("C:/Users/DKC/Desktop/Python Server Processing Code/Excel Files/fob.xlsx")


class SingleFabricHandler:
    def __init__(self, fabric_name):
        self.fabric_name = fabric_name
        self.processes = fabric_processes_list(self.fabric_name)


    def shiffly_handler(self):
            
            df_workflow_tracker = add_to_dataframe(
            df_workflow_tracker,
            stage="Submission",
            fabric_name=fabric,
            process="SHIFFLY",
            description=f"SHIFFLY process for {fabric}",
            quantity=total_fabric_quantity,
            color="null",
            start_date=start_date_china_shiffly,
            lowest_days=lowest_days_needed,
            end_date=end_date_china_shiffly
        ) 
    
     

if china_fabric_contains:
    stage = "FOB"

    #Repeating function, identifying china fabric again and again.
    def identify_china_fabric_name(data=user_submitted_data):
        list_china_fabrics = []
        for fabric in data["fabricDetails"]:
            if "china" in fabric["name"].lower():
                list_china_fabrics.append(fabric["name"])
        return list_china_fabrics

    china_fabric_names = identify_china_fabric_name() 
    china_fabric_helper_for_indian_fabric = china_fabric_names[0] #Why are we taking just the first element. 

    for china_fabric_name in china_fabric_names:
        for fabric_details in user_submitted_data["fabricDetails"]: #Inefficient Loop, because we are looping through 2 loops that might will return the same thing, so, it's better change the way this data is being stored and processed.
            if china_fabric_name == fabric_details["name"]:
                for process in fabric_details["processes"]:
                    number_of_days_required_for_china_fabric = int(df_fob_days_supporter[df_fob_days_supporter["process"] == process]["minimum_days"].iloc[0])
                    start_date = user_submitted_data["orderDate"]
                    end_date = start_date + timedelta(days=number_of_days_required_for_china_fabric)
                    for color in fabric_details["colors"]: # We can create something that can handle all the color cases at once. 
                        df_workflow_tracker = add_to_dataframe(
                            df_workflow_tracker,
                            stage="FOB",
                            fabric_name=fabric_details["name"],
                            process=process,
                            description=f"{process} process for {fabric_details['name']}",
                            quantity=color["quantity"],
                            color=color["name"],
                            start_date=start_date,
                            lowest_days=number_of_days_required_for_china_fabric,
                            end_date=end_date
                        )

    def identify_indian_fabric_names(data=user_submitted_data):
        list_indian_fabrics = []
        for fabric in data["fabricDetails"]:
            if "china" not in fabric["name"].lower(): # We will store data in a way that china and domestic fabrics are stored seperately.
                list_indian_fabrics.append(fabric["name"])
        return list_indian_fabrics

    india_fabric_names = identify_indian_fabric_names()

    for indian_fabric_name in india_fabric_names:
        for fabric_details in user_submitted_data["fabricDetails"]:
            if indian_fabric_name == fabric_details["name"]: # Unnecessary loop here for matching the names of the fabrics.
                process_to_check = {"DYING", "SHIFFLY"} # We are checking if these 2 processes are occuring simaltaneously or not.  We can seperate such process seperately in the begining of the code itself.
                if process_to_check.issubset(fabric_details["processes"]):
                    # Handling SHIFFLY Case
                    number_of_days_required_for_shiffly_fabric = int(df_fob_days_supporter[df_fob_days_supporter["process"] == "SHIFFLY"]["minimum_days"].iloc[0])
                    start_date = df_workflow_tracker[(df_workflow_tracker["Fabric Name"] == china_fabric_names[0]) & (df_workflow_tracker["Stage"] == "FOB")]["Original End Date"].max() #This line of code has to be understood.
                    shiffly_end_date = start_date + timedelta(days=number_of_days_required_for_shiffly_fabric) # Check if there is any possibility to make a universal days calculator.
                    df_workflow_tracker = add_to_dataframe(
                        df_workflow_tracker,
                        stage="FOB",
                        fabric_name=fabric_details["name"],
                        process="SHIFFLY",
                        description=f"SHIFFLY process for {fabric_details['name']}",
                        quantity=sum(int(color["quantity"]) for color in fabric_details["colors"]),
                        color="null",
                        start_date=start_date,
                        lowest_days=number_of_days_required_for_shiffly_fabric,
                        end_date=shiffly_end_date
                    )

                    # Handling DYING Case only for fabrics that have both shiffly and dying. 
                    number_of_days_required_for_shiffly_dying_fabric = int(df_fob_days_supporter[df_fob_days_supporter["process"] == "DYING"]["minimum_days"].iloc[0])
                    for color in fabric_details["colors"]: #Again fetching colors for this fabric.
                        dying_start_date = shiffly_end_date
                        dying_end_date = dying_start_date + timedelta(days=number_of_days_required_for_shiffly_dying_fabric)
                        df_workflow_tracker = add_to_dataframe(
                            df_workflow_tracker,
                            stage="FOB",
                            fabric_name=fabric_details["name"],
                            process="DYING",
                            description=f"DYING SHIFFLY for {fabric_details['name']} - {color['name']}",
                            quantity=color["quantity"],
                            color=color["name"],
                            start_date=dying_start_date,
                            lowest_days=number_of_days_required_for_shiffly_dying_fabric,
                            end_date=dying_end_date
                        )
                else:
                    for process in fabric_details["processes"]: #We are sperated the fabrics with multiple process above already in the if condition.
                        number_of_days_required_for_india_fabric = int(df_fob_days_supporter[df_fob_days_supporter["process"] == process]["minimum_days"].iloc[0]) #Specially this code is repeating. We can make a function for this. 
                        start_date = df_workflow_tracker[(df_workflow_tracker["Fabric Name"] == china_fabric_names[0]) & (df_workflow_tracker["Stage"] == "FOB")]["Original End Date"].max() 
                        end_date = start_date + timedelta(days=number_of_days_required_for_india_fabric) #We will create a simple function for calculating all the end dates.
                        for color in fabric_details["colors"]: # A function for priniting everytime a color has to be printed.
                            df_workflow_tracker = add_to_dataframe(
                                df_workflow_tracker,
                                stage="FOB",
                                fabric_name=fabric_details["name"],
                                process=process,
                                description=f"{process} process for {fabric_details['name']}",
                                quantity=color["quantity"],
                                color=color["name"],
                                start_date=start_date,
                                lowest_days=number_of_days_required_for_india_fabric,
                                end_date=end_date
                            )

df_workflow_tracker


,Stage,Fabric Name,Process,Description,Quantity of Material,Color,Original Start Date,Lowest Days,Original End Date
0,Purchase,WINDOWPANE GAUZ,GREIGE,Buying WINDOWPANE GAUZ,12000.0,null,2024-09-03,10,2024-09-13
1,Purchase,CHINA LACE,GREIGE,Buying CHINA LACE,3840.0,null,2024-09-03,30,2024-10-03
2,Purchase,FP001,GREIGE,Buying FP001,6000.0,null,2024-09-03,20,2024-09-23
3,Submission,FP001,SHIFFLY,SHIFFLY process for FP001,12000.0,null,2024-09-04,10,2024-09-14
4,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,3000,Red,2024-09-03,7,2024-09-10
5,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,4000,Blue,2024-09-03,7,2024-09-10
6,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,2000,Green,2024-09-03,7,2024-09-10
7,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,3000,White,2024-09-03,7,2024-09-10
8,FOB,WINDOWPANE GAUZ,DYING,DYING process for WINDOWPANE GAUZ,3000,Red,2024-09-10,7,2024-09-17
9,FOB,WINDOWPANE GAUZ,DYING,DYING process for WINDOWPANE GAUZ,4000,Blue,2024-09-10,7,2024-09-17


In [26]:
# Handling Bulk Stage
df_bulk_days_supporter = pd.read_excel("C:/Users/DKC/Desktop/Python Server Processing Code/Excel Files/bulk.xlsx")

def get_bulk_days(process, quantity):
    # Filter the DataFrame based on the process
    process_data = df_bulk_days_supporter[df_bulk_days_supporter['process'] == process]
    
    # Iterate through the rows to find the matching quantity range
    for _, row in process_data.iterrows(): #There are multiple instances where we are iterating over the quantity again and again so, we will create a single function for all of them. 
        if row['minimum_quantity'] <= int(quantity) <= row['maximum_quantity']:
            return row['minimum_days']
    
    return "Quantity out of range or process not found"

for fabrics_details in user_submitted_data["fabricDetails"]:
    fabric_name = fabrics_details["name"]
    
    if "SHIFFLY" not in fabrics_details["processes"]:
        for process in fabrics_details["processes"]: #This is same for LAB DIP, FOB both. So, better to create a function on the top level.
            start_date = df_workflow_tracker[
                (df_workflow_tracker["Stage"] == "FOB") & 
                (df_workflow_tracker["Fabric Name"] == fabric_name) & 
                (df_workflow_tracker["Process"] == process)
            ]["Original End Date"].max()
            
            for color in fabrics_details["colors"]:
                lowest_days = get_bulk_days(process, color["quantity"]) 
                end_date = start_date + timedelta(days=lowest_days)
                df_workflow_tracker = add_to_dataframe(
                    df_workflow_tracker,
                    stage="BULK",
                    fabric_name=fabric_name,
                    process=process,
                    description=f"{process} process for {fabric_name}",
                    quantity=color["quantity"],
                    color=color["name"],
                    start_date=start_date,
                    lowest_days=lowest_days,
                    end_date=end_date
                )
    else:
        # Handle SHIFFLY and DYING processes
        if "DYING" in fabrics_details["processes"]:
            # Start SHIFFLY process
            shiffly_start_date = df_workflow_tracker[
                (df_workflow_tracker["Stage"] == "Submission") & 
                (df_workflow_tracker["Fabric Name"] == fabric_name) & 
                (df_workflow_tracker["Process"] == "SHIFFLY")
            ]["Original End Date"].max()  # Assuming approvalDate is the start date for SHIFFLY
            print(shiffly_start_date)
            shiffly_days = get_bulk_days("SHIFFLY", sum(int(color["quantity"]) for color in fabrics_details["colors"]))
            shiffly_end_date = shiffly_start_date + timedelta(days=shiffly_days)
            
            df_workflow_tracker = add_to_dataframe(
                df_workflow_tracker,
                stage="BULK",
                fabric_name=fabric_name,
                process="SHIFFLY",
                description=f"SHIFFLY process for {fabric_name}",
                quantity=sum(int(color["quantity"]) for color in fabrics_details["colors"]),
                color="null",
                start_date=shiffly_start_date,
                lowest_days=shiffly_days,
                end_date=shiffly_end_date
            )
            
            # Start DYING process for each color after 2 days of SHIFFLY end date
            dying_start_date = shiffly_start_date + timedelta(days=2)
            for color in fabrics_details["colors"]:
                dying_days = get_bulk_days("DYING", color["quantity"])
                dying_end_date = dying_start_date + timedelta(days=dying_days)
                
                df_workflow_tracker = add_to_dataframe(
                    df_workflow_tracker,
                    stage="BULK",
                    fabric_name=fabric_name,
                    process="DYING",
                    description=f"DYING SHIFFLY for {fabric_name} - {color['name']}",
                    quantity=color["quantity"],
                    color=color["name"],
                    start_date=dying_start_date,
                    lowest_days=dying_days,
                    end_date=dying_end_date
                )

#Here we can create a dying function and then we can accordingly we can figure out if something will go there or not.

df_workflow_tracker


2024-09-14 00:00:00


,Stage,Fabric Name,Process,Description,Quantity of Material,Color,Original Start Date,Lowest Days,Original End Date
0,Purchase,WINDOWPANE GAUZ,GREIGE,Buying WINDOWPANE GAUZ,12000.0,null,2024-09-03,10,2024-09-13
1,Purchase,CHINA LACE,GREIGE,Buying CHINA LACE,3840.0,null,2024-09-03,30,2024-10-03
2,Purchase,FP001,GREIGE,Buying FP001,6000.0,null,2024-09-03,20,2024-09-23
3,Submission,FP001,SHIFFLY,SHIFFLY process for FP001,12000.0,null,2024-09-04,10,2024-09-14
4,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,3000,Red,2024-09-03,7,2024-09-10
5,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,4000,Blue,2024-09-03,7,2024-09-10
6,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,2000,Green,2024-09-03,7,2024-09-10
7,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,3000,White,2024-09-03,7,2024-09-10
8,FOB,WINDOWPANE GAUZ,DYING,DYING process for WINDOWPANE GAUZ,3000,Red,2024-09-10,7,2024-09-17
9,FOB,WINDOWPANE GAUZ,DYING,DYING process for WINDOWPANE GAUZ,4000,Blue,2024-09-10,7,2024-09-17


In [27]:
#Handling TRIMS & ACESSORIES 

df_trims_days_supporter = pd.read_excel("C:/Users/DKC/Desktop/Python Server Processing Code/Excel Files/trims&acessories.xlsx")

def get_trims_days(trim_name):
    days = df_trims_days_supporter[df_trims_days_supporter["DISCREPTION"].str.upper() == trim_name.upper()]["Minimum Days"].max()
    if pd.isna(days):  # Check if the result is NaN (no match found)
        return 10
    return days

# This might be even combined with repetative days and name function.

for acessory in user_submitted_data["accessories"]:
    start_date= user_submitted_data["orderDate"] + timedelta(days=3)
    lowest_days_needed = get_trims_days(acessory["name"])
    end_date = start_date+timedelta(days=lowest_days_needed)
    df_workflow_tracker = add_to_dataframe(
        df_workflow_tracker,
        stage="TRIMS ACESSORIES",
        fabric_name="TRIMS",
        process="null",
        description=acessory["name"],
        quantity=None,
        color=None,
        start_date=start_date,
        lowest_days=lowest_days_needed,
        end_date=end_date
    )    
    
for acessory in user_submitted_data["trims"]:
    start_date= user_submitted_data["orderDate"] + timedelta(days=3)
    lowest_days_needed = get_trims_days(acessory["name"])
    end_date = start_date+timedelta(days=lowest_days_needed)
    df_workflow_tracker = add_to_dataframe(
        df_workflow_tracker,
        stage="TRIMS ACESSORIES",
        fabric_name="TRIMS",
        process="null",
        description=acessory["name"],
        quantity=None,
        color=None,
        start_date=start_date,
        lowest_days=lowest_days_needed,
        end_date=end_date
    )  


#Acessory and trims can be combined. 

In [28]:
#Handling FIT DATA
def find_international_fabrics(data=user_submitted_data):
    result = []
    for fabric in data["fabricDetails"]:
        if "china".upper() in fabric["name"].upper():
            result.append(fabric["name"])
    
    return result

#Many times already we have identified china fabric so now it has to be used in a function.

international_fabrics = find_international_fabrics()

international_fabric_in_house_date = df_workflow_tracker[(df_workflow_tracker["Fabric Name"].isin(international_fabrics)) & (df_workflow_tracker["Stage"]=="Purchase")]["Original End Date"].max()

start_date_fit_1 =  max(user_submitted_data["orderDate"],international_fabric_in_house_date)
lowest_days_fit = 7
end_date_fit_1 = start_date_fit_1+timedelta(days=lowest_days_fit)

#End Date should be calculated with the help of a single universal function

df_workflow_tracker = add_to_dataframe(
        df_workflow_tracker,
        stage="FIT",
        fabric_name=None,
        process=None,
        description="FIT DKC",
        quantity=None,
        color=None,
        start_date=start_date_fit_1, 
        lowest_days=lowest_days_fit,
        end_date=end_date_fit_1
    ) 


start_date_fit_2 =   end_date_fit_1
end_date_fit_2 =  end_date_fit_1 + timedelta(days=lowest_days_fit)

df_workflow_tracker = add_to_dataframe(
        df_workflow_tracker,
        stage="FIT",
        fabric_name=None,
        process=None,
        description="FIT MERRIE APPROVAL",
        quantity=None,
        color=None,
        start_date=start_date_fit_2, 
        lowest_days=lowest_days_fit,
        end_date=end_date_fit_2
    ) 



# Both FIT logic can be encapsulated into 1

In [29]:
#Handling PP Data

fob_end_max_date = df_workflow_tracker[df_workflow_tracker["Stage"]=="FOB"]["Original End Date"].max()
fit_end_max_date =  df_workflow_tracker[df_workflow_tracker["Stage"]=="FIT"]["Original End Date"].max()

#This PP section can be somewhat inherited from the FIT section.
pp_start_date_1 = max(fob_end_max_date,fit_end_max_date)
pp_days_1 = 3
pp_end_date_1 = pp_start_date_1 + timedelta(days=pp_days_1)

df_workflow_tracker = add_to_dataframe(
        df_workflow_tracker,
        stage="PP",
        fabric_name=None,
        process=None,
        description="PP DKC",
        quantity=None,
        color=None,
        start_date=pp_start_date_1, 
        lowest_days=pp_days_1,
        end_date=pp_end_date_1
    ) 

pp_start_date_2 = pp_end_date_1
pp_days_2 = 7
pp_end_date_2 = pp_start_date_2 + timedelta(days=pp_days_2)

df_workflow_tracker = add_to_dataframe(
        df_workflow_tracker,
        stage="PP",
        fabric_name=None,
        process=None,
        description="PP MERRIE APPROVAL",
        quantity=None,
        color=None,
        start_date=pp_start_date_2, 
        lowest_days=pp_days_2,
        end_date=pp_end_date_2
    ) 


In [30]:
# Handling FABRIC AUDIT 

days_for_fabric_audit = 1

# Handle Fabric Audit section
for fabric in user_submitted_data["fabricDetails"]:
    fabric_name = fabric["name"]
    processes = fabric["processes"]
    colors = fabric["colors"]
    
    if "SHIFFLY" in processes:
        # Prioritize SHIFFLY and create audit entries for it  # I believe this is similar to our fob code. 
        for color in colors:
            color_name = color["name"]
            last_process_end_date = df_workflow_tracker[
                (df_workflow_tracker["Fabric Name"] == fabric_name) &
                (df_workflow_tracker["Color"] == color_name) & 
                (df_workflow_tracker["Stage"] == "BULK")
            ]["Original End Date"].max()
            start_date = last_process_end_date
            end_date = start_date + timedelta(days=1)
            description = f"{fabric_name} SHIFFLY {color_name}"
            df_workflow_tracker = add_to_dataframe(
                df_workflow_tracker,
                stage="FABRIC AUDIT",
                fabric_name=fabric_name,
                process="AUDIT",
                description=description,
                quantity=color["quantity"],
                color=color_name,
                start_date=start_date,
                lowest_days=1,
                end_date=end_date
            )
    else:
        # If SHIFFLY is not present, create audit entries for other processes
        for process in processes:
            for color in colors:
                color_name = color["name"]
                last_process_end_date = df_workflow_tracker[
                    (df_workflow_tracker["Fabric Name"] == fabric_name) &
                    (df_workflow_tracker["Color"] == color_name) & 
                    (df_workflow_tracker["Stage"] == "BULK")
                ]["Original End Date"].max()
                start_date = last_process_end_date
                end_date = start_date + timedelta(days=1)
                description = f"{fabric_name} {process} {color_name}"
                df_workflow_tracker = add_to_dataframe(
                    df_workflow_tracker,
                    stage="FABRIC AUDIT",
                    fabric_name=fabric_name,
                    process="AUDIT",
                    description=description,
                    quantity=color["quantity"],
                    color=color_name,
                    start_date=start_date,
                    lowest_days=1,
                    end_date=end_date
                )

    #These both can be combined and we can use function reuseability. 




In [31]:
#Handling Production Cutting Data
df_production_cutting_supporter = pd.read_excel("C:/Users/DKC/Desktop/Python Server Processing Code/Excel Files/production_cutting.xlsx")

def get_production_cutting_days(quantity):
    
    # Iterate through the rows to find the matching quantity range
    for _, row in df_production_cutting_supporter.iterrows():
        if row['minimum_quantity'] <= quantity <= row['maximum_quantity']:
            return row['minimum_days']
    
    return "Quantity out of range or fabric not found"


garments = user_submitted_data["garments"]

def production_cutting_time_calculator(fabric_name,data=user_submitted_data):
     pp_max_date = df_workflow_tracker[df_workflow_tracker["Stage"]=="PP"]["Original End Date"].max()
     fab_audit_max_date =  df_workflow_tracker[(df_workflow_tracker["Stage"]=="FABRIC AUDIT") & (df_workflow_tracker["Fabric Name"]==fabric_name)]["Original End Date"].max()
     return max(pp_max_date,fab_audit_max_date)



for garment, fabrics in garments.items():
        start_date_cutting = datetime.datetime(1970, 1, 24)
        for fabric in fabrics:
             start_date_cutting = max(start_date_cutting,production_cutting_time_calculator(fabric))

        lowest_days_needed_for_cutting = get_production_cutting_days(total_order_quantity)
        end_date =  start_date_cutting + datetime.timedelta(days=lowest_days_needed_for_cutting)
        df_workflow_tracker = add_to_dataframe(
                        df_workflow_tracker,
                        stage="PRODUCTION",
                        fabric_name=garment,
                        process="CUTTING",
                        description=None,
                        quantity=total_order_quantity,
                        color=None,
                        start_date=start_date_cutting,
                        lowest_days=lowest_days_needed_for_cutting,
                        end_date=end_date
                    )


TypeError: '<=' not supported between instances of 'int' and 'function'

In [15]:
df_production_stiching_supporter = pd.read_excel("C:/Users/DKC/Desktop/Python Server Processing Code/Excel Files/production_stiching.xlsx")

def get_production_stiching_days(quantity):
    
    # Iterate through the rows to find the matching quantity range
    for _, row in df_production_stiching_supporter.iterrows():
        if row['minimum_quantity'] <= quantity <= row['maximum_quantity']:
            return row['minimum_days']
    
    return "Quantity out of range or fabric not found"
# Process stitching stage
# Ensure garments are processed in the order of their cutting start dates
cutting_start_dates = {garment: df_workflow_tracker[
    (df_workflow_tracker["Stage"] == "PRODUCTION") &
    (df_workflow_tracker["Fabric Name"] == garment) &
    (df_workflow_tracker["Process"] == "CUTTING")
]["Original Start Date"].max() for garment in garments.keys()}

sorted_cutting_start_dates = sorted(cutting_start_dates.items(), key=lambda x: x[1])

for i, (garment, cutting_end_date) in enumerate(sorted_cutting_start_dates):
    if i == 0:
        # The first garment's stitching starts one day after its cutting ends
        start_date_stitching = cutting_end_date + timedelta(days=1)
        description = garment  # Single garment
    else:
        # The stitching for the combined garments starts one day after the cutting of the last garment
        last_garment_cutting_end_date = sorted_cutting_start_dates[i-1][1]
        start_date_stitching = last_garment_cutting_end_date + timedelta(days=1)
        description = f"{sorted_cutting_start_dates[i-1][0]} + {garment}"  # Combined garment description

    # Assuming stitching takes the same amount of days as cutting for simplicity
    lowest_days_needed_for_stitching = get_production_stiching_days(total_order_quantity)
    end_date_stitching = start_date_stitching + timedelta(days=lowest_days_needed_for_stitching)

    df_workflow_tracker = add_to_dataframe(
        df_workflow_tracker,
        stage="PRODUCTION",
        fabric_name=description,
        process="STITCHING",
        description=None,
        quantity=total_order_quantity,
        color=None,
        start_date=start_date_stitching,
        lowest_days=lowest_days_needed_for_stitching,
        end_date=end_date_stitching
    )


In [16]:
#Handling Initial Stage

initial_start_date_1 = df_workflow_tracker[df_workflow_tracker["Process"]=="STITCHING"]["Original Start Date"].max() + timedelta(days=2)

initial_days_needed =2

initial_end_date_1 = initial_start_date_1 + timedelta(days=initial_days_needed)

df_workflow_tracker = add_to_dataframe(
        df_workflow_tracker,
        stage="INITIAL",
        fabric_name=None,
        process=None,
        description="INITIAL 1",
        quantity=total_order_quantity,
        color=None,
        start_date=initial_start_date_1,
        lowest_days=initial_days_needed,
        end_date=initial_end_date_1
    )



initial_start_date_2 = initial_end_date_1 +timedelta(days=2)
initial_end_date_2 = initial_start_date_2 + timedelta(days=initial_days_needed)

df_workflow_tracker = add_to_dataframe(
        df_workflow_tracker,
        stage="INITIAL",
        fabric_name=None,
        process=None,
        description="INITIAL 2",
        quantity=total_order_quantity,
        color=None,
        start_date=initial_start_date_2,
        lowest_days=initial_days_needed,
        end_date=initial_end_date_2
    )


#These are simple logics, that can be handled through a single function.

In [17]:
#MID INSPECTION

mid_start_date_1 = df_workflow_tracker[df_workflow_tracker["Description"]=="INITIAL 2"]["Original End Date"].max() + timedelta(days=2)
print(mid_start_date_1)

mid_lowest_days = 3

df_workflow_tracker = add_to_dataframe(
        df_workflow_tracker,
        stage="MID INSPECTION",
        fabric_name=None,
        process=None,
        description="MID 1",
        quantity=total_order_quantity,
        color=None,
        start_date=mid_start_date_1,
        lowest_days=mid_lowest_days,
        end_date=mid_start_date_1 +  timedelta(days=mid_lowest_days)
    )


for i in range(2,5):
    start_date = df_workflow_tracker[df_workflow_tracker["Description"]==f"MID {i-1}"]["Original End Date"].max() + timedelta(days=3)
    end_date = start_date+ timedelta(days=mid_lowest_days)
    df_workflow_tracker = add_to_dataframe(
        df_workflow_tracker,
        stage="MID INSPECTION",
        fabric_name=None,
        process=None,
        description=f"MID {i}",
        quantity=total_order_quantity,
        color=None,
        start_date=start_date,
        lowest_days=mid_lowest_days,
        end_date=end_date
    )
    

2024-11-07 00:00:00


In [18]:
# Handling TOP/WEB/SIZE SET

start_date_size_set = df_workflow_tracker[df_workflow_tracker["Description"] == "PP MERRIE APPROVAL"]["Original End Date"].max()
lowest_days_needed = 3
end_date_size_set =  start_date_size_set + timedelta(days=lowest_days_needed)

df_workflow_tracker = add_to_dataframe(
    df_workflow_tracker,
    stage="SIZE SET/ TOP / WEB DATES",
    fabric_name=None,
    process=None,
    description="SIZE SET",
    quantity=total_order_quantity,
    color=None,
    start_date=start_date_size_set,
    lowest_days=lowest_days_needed,
    end_date=end_date_size_set
)


start_date_top = df_workflow_tracker[df_workflow_tracker["Stage"] == "INITIAL"]["Original End Date"].max()
lowest_days_needed = 1
end_date_top = start_date_top + timedelta(days=lowest_days_needed)

df_workflow_tracker = add_to_dataframe(
    df_workflow_tracker,
    stage="SIZE SET/ TOP / WEB DATES",
    fabric_name=None,
    process=None,
    description="TOP",
    quantity=total_order_quantity,
    color=None,
    start_date=start_date_top,
    lowest_days=lowest_days_needed,
    end_date=end_date_top
)

start_date_web = df_workflow_tracker[df_workflow_tracker["Stage"] == "INITIAL"]["Original End Date"].max()
lowest_days_needed = 1
end_date_web = start_date_top + timedelta(days=lowest_days_needed)

all_colors_for_web = set()

for fabric_details in user_submitted_data["fabricDetails"]:
    for fabric in fabric_details["colors"]:
        all_colors_for_web.add(fabric["name"])

all_colors_for_web

for  color in all_colors_for_web:
    df_workflow_tracker = add_to_dataframe(
    df_workflow_tracker,
    stage="SIZE SET/ TOP / WEB DATES",
    fabric_name=None,
    process=None,
    description="WEB",
    quantity=total_order_quantity,
    color=color,
    start_date=start_date_web,
    lowest_days=lowest_days_needed,
    end_date=end_date_web
)

df_workflow_tracker



#The logic is somewhat same as color logic all of the above. 

,Stage,Fabric Name,Process,Description,Quantity of Material,Color,Original Start Date,Lowest Days,Original End Date
0,Purchase,WINDOWPANE GAUZ,GREIGE,Buying WINDOWPANE GAUZ,12000.0,null,2024-09-03,10,2024-09-13
1,Purchase,CHINA LACE,GREIGE,Buying CHINA LACE,3840.0,null,2024-09-03,30,2024-10-03
2,Purchase,FP001,GREIGE,Buying FP001,6000.0,null,2024-09-03,20,2024-09-23
3,Submission,FP001,SHIFFLY,SHIFFLY process for FP001,12000.0,null,2024-09-04,10,2024-09-14
4,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,3000,Red,2024-09-03,7,2024-09-10
5,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,4000,Blue,2024-09-03,7,2024-09-10
6,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,2000,Green,2024-09-03,7,2024-09-10
7,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,3000,White,2024-09-03,7,2024-09-10
8,FOB,WINDOWPANE GAUZ,DYING,DYING process for WINDOWPANE GAUZ,3000,Red,2024-09-10,7,2024-09-17
9,FOB,WINDOWPANE GAUZ,DYING,DYING process for WINDOWPANE GAUZ,4000,Blue,2024-09-10,7,2024-09-17


In [19]:
#Handling  Final - DKC Final 1 or 2. 

dkc_final_start_date = df_workflow_tracker[df_workflow_tracker["Description"]=="MID 4"]['Original End Date'].max() + timedelta(days=1)
dkc_final_lowest_days = 1
dkc_final_end_date = dkc_final_start_date + timedelta(days=dkc_final_lowest_days)

df_workflow_tracker = add_to_dataframe(
    df_workflow_tracker,
    stage="Final",
    fabric_name=None,
    process=None,
    description="DKC FINAL",
    quantity=total_order_quantity,
    color=None,
    start_date=dkc_final_start_date,
    lowest_days=dkc_final_lowest_days,
    end_date=dkc_final_end_date
)

In [20]:
# X Factory final Date

x_factory_final_start_day = df_workflow_tracker[df_workflow_tracker['Description']=="DKC FINAL"]["Original End Date"].max()

x_factory_final_lowest_days = 1

x_factory_final_end_date = x_factory_final_start_day+timedelta(days=x_factory_final_lowest_days)

df_workflow_tracker = add_to_dataframe(
    df_workflow_tracker,
    stage="Final",
    fabric_name=None,
    process=None,
    description="X FACTORY FINAL Date",
    quantity=total_order_quantity,
    color=None,
    start_date=x_factory_final_start_day,
    lowest_days=x_factory_final_lowest_days,
    end_date=x_factory_final_end_date
)

df_workflow_tracker


#DKC final and X Factory dates can be clubbed together to get one unified function. 

,Stage,Fabric Name,Process,Description,Quantity of Material,Color,Original Start Date,Lowest Days,Original End Date
0,Purchase,WINDOWPANE GAUZ,GREIGE,Buying WINDOWPANE GAUZ,12000.0,null,2024-09-03,10,2024-09-13
1,Purchase,CHINA LACE,GREIGE,Buying CHINA LACE,3840.0,null,2024-09-03,30,2024-10-03
2,Purchase,FP001,GREIGE,Buying FP001,6000.0,null,2024-09-03,20,2024-09-23
3,Submission,FP001,SHIFFLY,SHIFFLY process for FP001,12000.0,null,2024-09-04,10,2024-09-14
4,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,3000,Red,2024-09-03,7,2024-09-10
5,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,4000,Blue,2024-09-03,7,2024-09-10
6,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,2000,Green,2024-09-03,7,2024-09-10
7,FOB,CHINA LACE,DYING,DYING process for CHINA LACE,3000,White,2024-09-03,7,2024-09-10
8,FOB,WINDOWPANE GAUZ,DYING,DYING process for WINDOWPANE GAUZ,3000,Red,2024-09-10,7,2024-09-17
9,FOB,WINDOWPANE GAUZ,DYING,DYING process for WINDOWPANE GAUZ,4000,Blue,2024-09-10,7,2024-09-17


In [21]:
df_workflow_tracker.to_csv("C:/Users/DKC/Desktop/Python Server Processing Code/china_case.csv")